# TechQuake: Behavior of Major Tech Stocks During Market Shocks

This notebook explores the behavior of selected technology-related stocks:

- AAPL (Apple)
- MSFT (Microsoft)
- NVDA (Nvidia)
- AMZN (Amazon)
- SPOT (Spotify)

Using daily price data from Yahoo Finance (`yfinance`), the notebook:

1. Fetches historical price data for the selected tickers.
2. Cleans and transforms the data using helper functions in `stock_data/`.
3. Computes daily returns, cumulative returns (indexed to 100), rolling volatility, and drawdowns.
4. Visualizes performance and risk, with an emphasis on market shocks such as the COVID-19 period and the 2022 inflation-driven selloff.

The goal is to compare how different tech companies behave under stress, both in terms of long-term growth and downside risk.

In [1]:
# Add the parent directory to the system path for proper module imports
import sys
sys.path.append("..")

In [2]:
# Project imports: data loading, processing, and plotting
from stock_data.fetch_data import fetch_stock_data
from stock_data.process_data import (
    extract_price_table,
    compute_daily_returns,
    compute_cumulative_returns,
    compute_rolling_volatility,
    compute_drawdowns,
)
from stock_data.visualizations import (
    plot_cumulative_returns,
    plot_stock_comparison,
    plot_volatility_with_events,
)

from datetime import datetime

In [ ]:
# Define tickers and start date
tickers = ["AAPL", "MSFT", "NVDA", "AMZN", "SPOT"]
start_date = "2021-01-01"

# Fetch the data
raw_data = fetch_stock_data(tickers, start_date=start_date)

# Display the fetched data
raw_data.head()

In [ ]:
# Extract a clean table of closing prices
prices = extract_price_table(raw_data, price_field="Close")
prices.head()

In [ ]:
# Compute daily returns for each ticker
daily_returns = compute_daily_returns(prices)
daily_returns.head()

In [ ]:
# Compute cumulative returns, starting from an index level of 100
cumulative_returns = compute_cumulative_returns(daily_returns, base=100.0)
cumulative_returns.head()

In [ ]:
# Describe how rolling volatility is calculated and what it represents. It shows how the stock price fluctuates over time.
rolling_volatility = compute_rolling_volatility(daily_returns, window=21)
rolling_volatility.tail()